In [15]:
pip install pandas-categorical

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/numpy-1.25.2.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pandas as pd
import pandas_categorical as pdc

# read dataframe

In [17]:
# optimal column types
SUB_DTYPES = {
    'Invoice/Item Number': "string",
    'Date': 'datetime64[ns]',
    'Store Number': "uint16",
    'Store Name': "string",
    'Address': 'string',
    'City': "string",
    'Zip Code': 'string',
    'Store Location': 'string',
    'County Number': "Int16",
    'County': "string",
    'Category': "Int32",
    'Category Name': 'string',
    'Vendor Number': "Int16",
    'Vendor Name': 'string',
    'Item Number': "string",
    'Item Description': "string",
    'Pack': "uint16",
    'Bottle Volume (ml)': "uint32",
    'State Bottle Cost': float,
    'State Bottle Retail': float,
    'Bottles Sold': int,
    'Sale (Dollars)': float,
    'Volume Sold (Liters)': float,
    'Volume Sold (Gallons)': float,
}

In [18]:
df = pd.read_csv(
    "/kaggle/input/big-sales-data/Sales_Data/Liquor_Sales.csv",
    engine='pyarrow',
    dtype=SUB_DTYPES
)

In [19]:
# Let's put a small part of the data aside
(_, df), (_, df_1) = df.groupby(df.index < 1000)

# Compression by categorization

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19665763 entries, 1000 to 19666762
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Invoice/Item Number    string        
 1   Date                   datetime64[ns]
 2   Store Number           uint16        
 3   Store Name             string        
 4   Address                string        
 5   City                   string        
 6   Zip Code               string        
 7   Store Location         string        
 8   County Number          Int16         
 9   County                 string        
 10  Category               Int32         
 11  Category Name          string        
 12  Vendor Number          Int16         
 13  Vendor Name            string        
 14  Item Number            string        
 15  Item Description       string        
 16  Pack                   uint16        
 17  Bottle Volume (ml)     uint32        
 18  State Bottle Cost     

In [21]:
CAT_COLS = [
    'Date',
    'Store Number',
    'Store Name',
    'Address',
    'City',
    'Zip Code',
    'Store Location',
    'County Number',
    'County',
    'Category',
    'Category Name',
    'Vendor Number',
    'Vendor Name',
    'Item Number',
    'Item Description',
    'Pack',
    'Bottle Volume (ml)',
]
ORDERED_COLS = [
    'Date',
    'Bottle Volume (ml)',
]

pdc.cat_astype(df, cat_cols=CAT_COLS, sub_dtypes=SUB_DTYPES, ordered_cols=ORDERED_COLS)
pdc.cat_astype(df_1, cat_cols=CAT_COLS, sub_dtypes=SUB_DTYPES, ordered_cols=ORDERED_COLS)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19665763 entries, 1000 to 19666762
Data columns (total 24 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   Invoice/Item Number    string  
 1   Date                   category
 2   Store Number           category
 3   Store Name             category
 4   Address                category
 5   City                   category
 6   Zip Code               category
 7   Store Location         category
 8   County Number          category
 9   County                 category
 10  Category               category
 11  Category Name          category
 12  Vendor Number          category
 13  Vendor Name            category
 14  Item Number            category
 15  Item Description       category
 16  Pack                   category
 17  Bottle Volume (ml)     category
 18  State Bottle Cost      float64 
 19  State Bottle Retail    float64 
 20  Bottles Sold           int64   
 21  Sale (Dollars)         float64 

# Saving of categorical types in concat

In [23]:
df = pdc.concat_categorical([df, df_1], ignore_index=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19666763 entries, 0 to 19666762
Data columns (total 24 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   Invoice/Item Number    string  
 1   Date                   category
 2   Store Number           category
 3   Store Name             category
 4   Address                category
 5   City                   category
 6   Zip Code               category
 7   Store Location         category
 8   County Number          category
 9   County                 category
 10  Category               category
 11  Category Name          category
 12  Vendor Number          category
 13  Vendor Name            category
 14  Item Number            category
 15  Item Description       category
 16  Pack                   category
 17  Bottle Volume (ml)     category
 18  State Bottle Cost      float64 
 19  State Bottle Retail    float64 
 20  Bottles Sold           int64   
 21  Sale (Dollars)         float6

# Saving categorical types when merging

In [25]:
another_df = pd.read_csv("/kaggle/input/big-sales-data/Sales_Data/Sales_April_2019.csv",
                         usecols=['Order Date', 'Order ID'])
another_df = another_df[another_df['Order Date'] != "Order Date"]
another_df['Order Date'] = another_df['Order Date'].astype('datetime64[ns]').dt.normalize()
pdc.cat_astype(
    another_df,
    cat_cols=['Order Date'],
    sub_dtypes={"Order Date": 'datetime64[ns]'},
    ordered_cols=['Order Date'],
)

/tmp/ipykernel_32/3806307031.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  another_df['Order Date'] = another_df['Order Date'].astype('datetime64[ns]').dt.normalize()


In [26]:
another_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18348 entries, 0 to 18382
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Order ID    18289 non-null  object  
 1   Order Date  18289 non-null  category
dtypes: category(1), object(1)
memory usage: 305.9+ KB


In [27]:
res = pdc.merge_categorical(
    df,
    another_df.head(1000),
    left_on='Date',
    right_on='Order Date',
    how='left'
)

/tmp/ipykernel_32/1231945684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res = pdc.merge_categorical(


In [28]:
# The categorical type of the date column is NOT lost
res.dtypes

Invoice/Item Number      string[python]
Date                           category
Store Number                   category
Store Name                     category
Address                        category
City                           category
Zip Code                       category
Store Location                 category
County Number                  category
County                         category
Category                       category
Category Name                  category
Vendor Number                  category
Vendor Name                    category
Item Number                    category
Item Description               category
Pack                           category
Bottle Volume (ml)             category
State Bottle Cost               float64
State Bottle Retail             float64
Bottles Sold                      int64
Sale (Dollars)                  float64
Volume Sold (Liters)            float64
Volume Sold (Gallons)           float64
Order ID                         object
